![](https://memesbams.com/wp-content/uploads/2017/11/sheldon-sarcasm-meme.jpg)

https://www.kaggle.com/danofer/sarcasm
<div class="markdown-converter__text--rendered"><h3>Context</h3>

<p>This dataset contains 1.3 million Sarcastic comments from the Internet commentary website Reddit. The dataset was generated by scraping comments from Reddit (not by me :)) containing the <code>\s</code> ( sarcasm) tag. This tag is often used by Redditors to indicate that their comment is in jest and not meant to be taken seriously, and is generally a reliable indicator of sarcastic comment content.</p>

<h3>Content</h3>

<p>Data has balanced and imbalanced (i.e true distribution) versions. (True ratio is about 1:100). The
corpus has 1.3 million sarcastic statements, along with what they responded to as well as many non-sarcastic comments from the same source.</p>

<p>Labelled comments are in the <code>train-balanced-sarcasm.csv</code> file.</p>

<h3>Acknowledgements</h3>

<p>The data was gathered by: Mikhail Khodak and Nikunj Saunshi and Kiran Vodrahalli for their article "<a href="https://arxiv.org/abs/1704.05579" rel="nofollow">A Large Self-Annotated Corpus for Sarcasm</a>". The data is hosted <a href="http://nlp.cs.princeton.edu/SARC/0.0/" rel="nofollow">here</a>.</p>

<p>Citation:</p>

<pre><code>@unpublished{SARC,
  authors={Mikhail Khodak and Nikunj Saunshi and Kiran Vodrahalli},
  title={A Large Self-Annotated Corpus for Sarcasm},
  url={https://arxiv.org/abs/1704.05579},
  year=2017
}
</code></pre>

<p><a href="http://nlp.cs.princeton.edu/SARC/0.0/readme.txt" rel="nofollow">Annotation of files in the original dataset: readme.txt</a>.</p>

<h3>Inspiration</h3>

<ul>
<li>Predicting sarcasm and relevant NLP features (e.g. subjective determinant, racism, conditionals, sentiment heavy words, "Internet Slang" and specific phrases). </li>
<li>Sarcasm vs Sentiment</li>
<li>Unusual linguistic features such as caps, italics, or elongated words. e.g., "Yeahhh, I'm sure THAT is the right answer".</li>
<li>Topics that people tend to react to sarcastically</li>
</ul></div>

In [ ]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp

In [1]:
import sys
import time
import sparknlp

from pyspark.sql import SparkSession
packages = [
    'JohnSnowLabs:spark-nlp: 2.5.5'
]
spark = SparkSession \
    .builder \
    .appName("ML SQL session") \
    .config('spark.jars.packages', ','.join(packages)) \
    .config('spark.executor.instances','2') \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory","16g") \
    .getOrCreate()

In [2]:
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:   2.4.2
Apache Spark version:  2.4.4


In [3]:
! wget -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sarcasm/train-balanced-sarcasm.csv -P /tmp

--2020-02-11 19:18:09--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sarcasm/train-balanced-sarcasm.csv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.237.229
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.237.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255268960 (243M) [text/csv]
Saving to: ‘/tmp/train-balanced-sarcasm.csv’

train-balanced-sarc 100%[===================>] 243,44M  5,01MB/s    in 33s     

2020-02-11 19:18:43 (7,46 MB/s) - ‘/tmp/train-balanced-sarcasm.csv’ saved [255268960/255268960]



In [4]:
from pyspark.sql import SQLContext

sql = SQLContext(spark)

trainBalancedSarcasmDF = spark.read.option("header", True).option("inferSchema", True).csv("/tmp/train-balanced-sarcasm.csv")
trainBalancedSarcasmDF.printSchema()

# Let's create a temp view (table) for our SQL queries
trainBalancedSarcasmDF.createOrReplaceTempView('data')

sql.sql('SELECT COUNT(*) FROM data').collect()

root
 |-- label: integer (nullable = true)
 |-- comment: string (nullable = true)
 |-- author: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- score: string (nullable = true)
 |-- ups: string (nullable = true)
 |-- downs: string (nullable = true)
 |-- date: string (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- parent_comment: string (nullable = true)



[Row(count(1)=1010826)]

In [5]:
sql.sql('select * from data limit 20').show()

+-----+--------------------+------------------+------------------+-----+---+-----+-------+-------------------+--------------------+
|label|             comment|            author|         subreddit|score|ups|downs|   date|        created_utc|      parent_comment|
+-----+--------------------+------------------+------------------+-----+---+-----+-------+-------------------+--------------------+
|    0|          NC and NH.|         Trumpbart|          politics|    2| -1|   -1|2016-10|2016-10-16 23:55:23|Yeah, I get that ...|
|    0|You do know west ...|         Shbshb906|               nba|   -4| -1|   -1|2016-11|2016-11-01 00:24:10|The blazers and M...|
|    0|They were underdo...|          Creepeth|               nfl|    3|  3|    0|2016-09|2016-09-22 21:45:37|They're favored t...|
|    0|"This meme isn't ...|         icebrotha|BlackPeopleTwitter|   -8| -1|   -1|2016-10|2016-10-18 21:03:47|deadass don't kil...|
|    0|I could use one o...|         cush2push|MaddenUltimateTeam|    6| -1|

In [6]:
sql.sql('select label,count(*) as cnt from data group by label order by cnt desc').show()

+-----+------+
|label|   cnt|
+-----+------+
|    0|505413|
|    1|505413|
+-----+------+



In [7]:
sql.sql('select count(*) from data where comment is null').collect()

[Row(count(1)=53)]

In [8]:
df = sql.sql('select label,concat(parent_comment,"\n",comment) as comment from data where comment is not null and parent_comment is not null limit 100000')
print(type(df))
df.printSchema()
df.show()

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- label: integer (nullable = true)
 |-- comment: string (nullable = true)

+-----+--------------------+
|label|             comment|
+-----+--------------------+
|    0|Yeah, I get that ...|
|    0|The blazers and M...|
|    0|They're favored t...|
|    0|deadass don't kil...|
|    0|Yep can confirm I...|
|    0|do you find arian...|
|    0|What's your weird...|
|    0|Probably Sephirot...|
|    0|What to upgrade? ...|
|    0|Probably count Ka...|
|    0|I bet if that mon...|
|    0|James Shields Wil...|
|    0|There's no time t...|
|    0|Team Specific Thr...|
|    0|Ill give you a hi...|
|    0|Star Wars, easy. ...|
|    0|You're adorable.
...|
|    0|He actually acts ...|
|    0|Clinton struggles...|
|    0|Is that the Older...|
+-----+--------------------+
only showing top 20 rows



In [9]:
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from pyspark.ml import Pipeline


document_assembler = DocumentAssembler() \
    .setInputCol("comment") \
    .setOutputCol("document")
    
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence") \
    .setUseAbbreviations(True)
    
tokenizer = Tokenizer() \
  .setInputCols(["sentence"]) \
  .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")
    
normalizer = Normalizer() \
    .setInputCols(["stem"]) \
    .setOutputCol("normalized")

finisher = Finisher() \
    .setInputCols(["normalized"]) \
    .setOutputCols(["ntokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(True)

nlp_pipeline = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, stemmer, normalizer, finisher])
nlp_model = nlp_pipeline.fit(df)
processed = nlp_model.transform(df).persist()
processed.count()
processed.show()

+-----+--------------------+--------------------+
|label|             comment|             ntokens|
+-----+--------------------+--------------------+
|    0|Yeah, I get that ...|[yeah, i, get, th...|
|    0|The blazers and M...|[the, blazer, and...|
|    0|They're favored t...|[theyr, favor, to...|
|    0|deadass don't kil...|[deadass, dont, k...|
|    0|Yep can confirm I...|[yep, can, confir...|
|    0|do you find arian...|[do, you, find, a...|
|    0|What's your weird...|[what, your, weir...|
|    0|Probably Sephirot...|[probabl, sephiro...|
|    0|What to upgrade? ...|[what, to, upgrad...|
|    0|Probably count Ka...|[probabl, count, ...|
|    0|I bet if that mon...|[i, bet, if, that...|
|    0|James Shields Wil...|[jame, shield, wi...|
|    0|There's no time t...|[there, no, time,...|
|    0|Team Specific Thr...|[team, specif, th...|
|    0|Ill give you a hi...|[ill, give, you, ...|
|    0|Star Wars, easy. ...|[star, war, easi,...|
|    0|You're adorable.
...|  [your, ador, note]|


In [10]:
train, test = processed.randomSplit(weights=[0.7, 0.3], seed=123)
print(train.count())
print(test.count())

70136
29864


In [11]:
from pyspark.ml import feature as spark_ft

stopWords = spark_ft.StopWordsRemover.loadDefaultStopWords('english')
sw_remover = spark_ft.StopWordsRemover(inputCol='ntokens', outputCol='clean_tokens', stopWords=stopWords)
tf = spark_ft.CountVectorizer(vocabSize=500, inputCol='clean_tokens', outputCol='tf')
idf = spark_ft.IDF(minDocFreq=5, inputCol='tf', outputCol='idf')

feature_pipeline = Pipeline(stages=[sw_remover, tf, idf])
feature_model = feature_pipeline.fit(train)

train_featurized = feature_model.transform(train).persist()
train_featurized.count()
train_featurized.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|             comment|             ntokens|        clean_tokens|                  tf|                 idf|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|              !
Goes|               [goe]|               [goe]|   (500,[375],[1.0])|(500,[375],[4.866...|
|    0|!completed
!compl...|  [complet, complet]|  [complet, complet]|   (500,[227],[2.0])|(500,[227],[8.875...|
|    0|""" ""Very Right ...|[veri, right, win...|[veri, right, win...|(500,[1,7,31,77,9...|(500,[1,7,31,77,9...|
|    0|""" Perhaps you n...|[perhap, you, ne,...|[perhap, ne, stro...|    (500,[34],[1.0])|(500,[34],[3.1336...|
|    0|""" This covering...|[thi, cover, not,...|[thi, cover, onli...|(500,[0,6,14,18,2...|(500,[0,6,14,18,2...|
|    0|"""*Kirk
I am sin...|[kirk, i, am, sin...|[kirk, singl, gue...|(500,[31,168,348]...|(500,

In [12]:
train_featurized.groupBy("label").count().show()
train_featurized.printSchema()

+-----+-----+
|label|count|
+-----+-----+
|    0|40466|
|    1|29670|
+-----+-----+

root
 |-- label: integer (nullable = true)
 |-- comment: string (nullable = true)
 |-- ntokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- clean_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)
 |-- idf: vector (nullable = true)



In [13]:
from pyspark.ml import classification as spark_cls

rf = spark_cls. RandomForestClassifier(labelCol="label", featuresCol="idf", numTrees=100)

model = rf.fit(train_featurized)

In [14]:
test_featurized = feature_model.transform(test)
preds = model.transform(test_featurized)
preds.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|             comment|             ntokens|        clean_tokens|                  tf|                 idf|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|!RemindMe 1 week
...|[remindm, week, r...|[remindm, week, r...|(500,[56,132],[1....|(500,[56,132],[3....|[58.8715006890861...|[0.58871500689086...|       0.0|
|    0|!Remindme 2 weeks...|[remindm, week, r...|[remindm, week, r...|   (500,[132],[2.0])|(500,[132],[8.254...|[58.8715006890861...|[0.58871500689086...|       0.0|
|    0|!SH!TPOST!: All t...|[shtpost, all, th...|[shtpost, poor, u...|(500,[286,476],[1...|(500,[286,476],[4...|[58.6927668196978...|[0.58692766819697...|       0.0|
|   

In [15]:
pred_df = preds.select('comment', 'label', 'prediction').toPandas()

In [16]:
pred_df.head()

,comment,label,prediction
0,!RemindMe 1 week\n!RemindMe 2 days,0,0.0
1,!Remindme 2 weeks\n!Remindme 2 weeks,0,0.0
2,!SH!TPOST!: All those poor USA Streamers\nNow ...,0,0.0
3,"""""""**FUCK** Cloud"""" - Link main""\nYep, that's ...",0,0.0
4,"""""""*Komrad\n""*""""Those were just pro-USA rebels...",0,0.0


In [17]:
import pandas as pd
from sklearn import metrics as skmetrics
pd.DataFrame(
    data=skmetrics.confusion_matrix(pred_df['label'], pred_df['prediction']),
    columns=['pred ' + l for l in ['0','1']],
    index=['true ' + l for l in ['0','1']]
)

,pred 0,pred 1
true 0,17122,102
true 1,12143,497


In [18]:
print(skmetrics.classification_report(pred_df['label'], pred_df['prediction'], 
                                      target_names=['0','1']))

              precision    recall  f1-score   support

           0       0.59      0.99      0.74     17224
           1       0.83      0.04      0.08     12640

    accuracy                           0.59     29864
   macro avg       0.71      0.52      0.41     29864
weighted avg       0.69      0.59      0.46     29864



In [19]:
spark.stop()